In [7]:
# model_id = "meta-llama/Llama-2-7b-chat-hf"
model_id = "meta-llama/Llama-2-13b-chat-hf"

In [8]:
number_of_shots = 5

In [9]:
nbit = 4
# llamasize = '7B'
llamasize = '13B'
temp = 0.1

In [10]:
experiment_name = "2023.12.06-05_LLama2_13B-chat-hf_4-bit_float-16_5-few-shot_SEMANTIC_SIMILARITY_WITH_DIFFERENT_CLASSES"

In [11]:
from google.colab import drive
from IPython.display import Image, display

mount_point = "/content/gdrive"
base_path = mount_point + "/MyDrive/Colab/SDG/data"
input_data_path = base_path + "/input/sdg_17_labels_classification_dataset_1020_texts_TEST.xlsx"
input_data_path_train = base_path + "/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx"

top7_train_data_sim_to_each_test_data_filename = base_path + "/input/top7_train_data_sim_to_each_test_data_but_with_different_classes.pickle"

output_data_path = base_path + "/../Llama-2/data/output/" + experiment_name
log_filename = base_path + "/../Llama-2/logs/" + experiment_name + ".log"

drive.mount(mount_point, force_remount=True)

Mounted at /content/gdrive


# LOGIN DA HUGGING FACE

In [12]:
# import os
# from IPython.display import clear_output
# os.environ["OPENAI_API_KEY"] = input("INSERT YOU OPENAI API KEY:")
# clear_output()

from IPython.display import clear_output
huggingface_cli_token = input("INSERT YOUR huggingface-cli_token:")
clear_output()

In [13]:
# @title
!huggingface-cli login --token $huggingface_cli_token

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [14]:
# @title
# from pathlib import Path

In [15]:
# @title
# # import logging

# # logger = logging.getLogger('my_logger')
# # # , level=logging.INFO,
# # #logging.basicConfig(filename=base_path + f"/../Llama-2/logs/{experiment_name}.log", filemode='a', format='%(asctime)s - %(levelname)s - %(message)s')
# # logging.basicConfig(filename="test.log", filemode='a', format='%(asctime)s - %(levelname)s - %(message)s')

# %%python

# import logging
# logger = logging.getLogger('my_logger')

# logging.basicConfig(
#     #filename=experiment_name+'.log', # write to this file
#     filename = 'app.log',
#     filemode='a', # open in append mode
#     level=logging.INFO,
#     format='%(name)s - %(levelname)s - %(message)s'
#     )

# # logging.warning('This will get logged to a file')
# logging.info('test')

In [16]:
# @title
# print(Path('app.log').read_text())

In [17]:
# @title
# from google.colab import files

# !mv app.log output_data_path

In [18]:
# @title
# import locale
# print(locale.getpreferredencoding())

In [19]:
# @title
# import locale
# def getpreferredencoding(do_setlocale = True):
#     return "UTF-8"
# locale.getpreferredencoding = getpreferredencoding

In [20]:
# @title
# !touch /content/gdrive/MyDrive/Colab/SDG/data/../Llama-2/logs/2023.12.06-1_LLama2_7B-chat-hf_4-bit_float-16_3-few-shot_SEMANTIC_SIMILARITY_WITH_DIFFERENT_CLASSES.log

In [21]:
# @title
# base_path + f"/../Llama-2/logs/{experiment_name}.log"

In [22]:
# @title
# output_data_path

In [23]:
# @title
# logging.info("test")
# logging.warning('This will get logged to a file')

In [24]:
# @title
# print(Path(base_path + f"/../Llama-2/logs/{experiment_name}.log").read_text())
# print(Path("test.log").read_text())

# INSTALLAZIONE E IMPORT LIBRERIE

In [25]:
# @title
import pickle

In [26]:
# @title
import warnings
warnings.filterwarnings("ignore")

In [27]:
# @title
with open(top7_train_data_sim_to_each_test_data_filename, 'rb') as f:
    top7_train_data_sim_to_each_test_data = pickle.load(f)

In [28]:
# @title
#!pip install --upgrade pip
!pip install -q transformers --progress-bar off # Download e caricamento modelli
!pip install -q bitsandbytes --progress-bar off # Quantizzazione
!pip install -q accelerate --progress-bar off

In [29]:
# @title
# Installazione "permanente"
!pip install -q sentencepiece --progress-bar off && touch ~/.ipython/profile_default/startup/00-first.py && echo "import <sentencepiece>" >> ~/.ipython/profile_default/startup/00-first.py

In [30]:
# @title
!pip install -q pillow==10.0.1 --progress-bar off
!pip install -q langchain[llms] --progress-bar off

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [31]:
# @title
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

from langchain.llms import HuggingFacePipeline
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate

In [32]:
# @title
import transformers
from transformers import LlamaForCausalLM, LlamaTokenizer, GenerationConfig # Questi moduli sono dedicati al modello LLama2
from transformers import AutoModelForCausalLM, AutoTokenizer # Questi moduli permettono di scaricare qualsiasi modello e tokenizer
import pandas as pd
import torch # PyTorch
import langchain

In [33]:
# @title
torch.cuda.empty_cache()

# DOWNLOAD E CARICAMENTO DEL MODELLO

In [34]:
# @title
# pip install --upgrade bitsandbytes

In [35]:
# @title
model = LlamaForCausalLM.from_pretrained(  # Caricamento del modello usando la libreria dedicata. In questo caso, "ForCausal" si riferisce al modello per la generazione di testo
    model_id, # Nome del modello precedentemente impostato
    load_in_4bit = True, # Quantizzazione in 4 bit (per 8 bit usare il parametro load_in_8bit)
    torch_dtype = torch.float16 # Impostare questa codifica aiuta il modello in fase di generazione, in quanto il prompt è tipicamente float16
)

config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [36]:
# @title
torch.cuda.empty_cache()

In [37]:
# @title
# !nvidia-smi

In [38]:
# @title
tokenizer = LlamaTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [39]:
# @title
torch.cuda.empty_cache()

In [40]:
# @title
# model.eval()

In [41]:
# @title
torch.cuda.empty_cache()

## TEST

In [42]:
# @title
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=200,  # max number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

In [43]:
# @title
torch.cuda.empty_cache()

In [44]:
# @title
llm = HuggingFacePipeline(pipeline=generate_text)

In [45]:
# @title
torch.cuda.empty_cache()

In [46]:
# @title
data = pd.read_excel(input_data_path)

In [47]:
# @title
# data.shape

In [48]:
# @title
# data.groupby('sdg').count()['text']

In [49]:
# @title
data_sel = data

In [50]:
# @title
# import pandas as pd
# data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0']).sample(2)
# data_sel_4_few_shot_examples

In [51]:
# @title
data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0'])

In [52]:
# @title
# if USE_SAME_FEW_SHOTS==True:
#     data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0'])
#     data_sel_4_few_shot_examples = data_sel_4_few_shot_examples[
#         (data_sel_4_few_shot_examples.text_id==5614) |
#         (data_sel_4_few_shot_examples.text_id==2089) |
#         (data_sel_4_few_shot_examples.text_id==5665) |
#         (data_sel_4_few_shot_examples.text_id==370)  |
#         (data_sel_4_few_shot_examples.text_id==1422) |
#         (data_sel_4_few_shot_examples.text_id==4329) |
#         (data_sel_4_few_shot_examples.text_id==4929)
#     ]
# else:
#     data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0']).sample(7)

# data_sel_4_few_shot_examples

In [53]:
# @title
# few_shots = ""

# for _, row in data_sel_4_few_shot_examples[['text', 'sdg']].iterrows():
#     input_text = "Input text: \""+row['text']+"\"\n"
#     output_text = "Output: \"---SDG "+str(row['sdg'])+'---\"\n\n'

#     few_shots += input_text
#     few_shots += output_text

# print(few_shots)

In [54]:
# @title
# template = """[INST]
# <<SYS>>
# Classify the following text between triple quotes according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015, choosing one (and only one) of the SDG labels based on it best representing the content of the entire text enclosed in triple quotes.

# In the following lines, a text enclosed in triple quotes will be given to you.

# The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

# The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

# If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

# The following are 7 examples of expected classification given an input text:

# """+few_shots+"""


# The text to be processed is:
# <</SYS>>

# '''
# {text}
# '''

# <<SYS>>
# Answer with label only usign the format:---LABEL---.
# <</SYS>>
# [/INST]
# """

In [55]:
# @title
# prompt = PromptTemplate(
#     input_variables=["text"],
#     template=template,
# )

In [56]:
# @title
torch.cuda.empty_cache()

In [57]:
# @title
# top7_train_data_sim_to_each_test_data

In [58]:
# file = open(log_filename, 'a+')
# file.write("ecchecaz")
# file.close()

# log_filename

In [59]:
data_sel = data.iloc[26:]

In [ ]:
%%time
#results = data_sel.apply(lambda x: llm(prompt.format(text=x["text"])), axis=1)

cnt = 26
results = []

for index, row in data_sel.iterrows():

  text_id = row['text_id']
  text = row["text"]
  sdg_true = row["sdg"]

  file = open(log_filename, 'a+', encoding='utf-8')

  print("\n\n##################################################")
  file.write("\n\n##################################################\n")

  print("EXPECTED_SDG:", row["sdg"], "TEXT:", row["text"].replace('\n',''),'\n\n')
  file.write(f"EXPECTED_SDG: {row['sdg']} TEXT: " + row['text'].replace('\n','') + "\n\n\n")

  few_shots_list = top7_train_data_sim_to_each_test_data[row['text_id']][:number_of_shots]

  few_shots = ""

  for train_data_id in few_shots_list:

    # print("##################################################")
    # print(train_data_id)

    train_row = data_sel_4_few_shot_examples[data_sel_4_few_shot_examples.text_id==train_data_id]

    # print(train_row)

    # print(train_row.text.values[0])
    # print(train_row.sdg.values[0])

    input_text = "Input text: \""+train_row.text.values[0].replace('\n','')+"\"\n"
    output_text = "Output: \"---SDG "+str(train_row.sdg.values[0])+'---\"\n\n'

    few_shots += input_text
    few_shots += output_text

  print(few_shots)
  file.write(few_shots)
  file.write('\n')


  template = """[INST]
<<SYS>>
Classify the following text between triple quotes according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015, choosing one (and only one) of the SDG labels based on it best representing the content of the entire text enclosed in triple quotes.

In the following lines, a text enclosed in triple quotes will be given to you.

The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

The following are """+str(number_of_shots)+""" non exhaustive examples of expected classification given an input text:

"""+few_shots+"""


The text to be processed is:
<</SYS>>

'''
{text}
'''

<<SYS>>
Answer with label only usign the format:---LABEL---.
<</SYS>>
[/INST]
"""
  try:

    prompt = PromptTemplate(
      input_variables=["text"],
      template=template,
    )

    # print(template)

    result = llm(prompt.format(text=text))
  except Exception as e:
    print(cnt, text_id, sdg_true, e)
    result = str(e)
    output_text = "Si è verificato un errore: \n("+str(cnt)+", "+str(text_id)+", "+str(sdg_true)+", "+str(e)+")\n"
    file.write(output_text)

  result_extended = (cnt, text_id, sdg_true, result)

  print(result_extended)
  file.write(str(result_extended)+'\n')

  with open(output_data_path+f'/sdg_classification_Llama2_13B_SEMANTIC_SIMILARITY_{cnt}.pkl', 'wb') as f:
      pickle.dump(result_extended, f)

  results.append(result_extended)

  cnt+=1
  file.close()
  # if cnt==5:
  #   break



Output streaming troncato alle ultime 5000 righe.
Output: "---SDG 16---"

Input text: "Overall, at national level, the authorities estimate that reaching the level of treatment which meets discharge standards will require levels of investment of some MXN114 billion (USD 9.5 billion). In addition, the municipalities of Chihuahua and Juarez have been active in recycling water for green areas’ irrigation and industrial use. This map is for illustrative purposes and is without prejudice to the status of or sovereignty over any territory covered by this map."
Output: "---SDG 6---"


(738, 5570, 12, '---SDG 12---')


##################################################
EXPECTED_SDG: 12 TEXT: Sound waste management is absolutely essential. M. Wallet (20101, Conlifcl In Campania:' Addilbnal inlomalion Irom David Newman, Presidenl ol the Hall UNEP/OCHA (2011). Rapid disaster waste manapemenl assessment 26 Documenls'Gnana. The overall reductbn from the waste sector between 1990 and 2006 amounts to

In [ ]:
# @title
# data_sel_4_few_shot_examples[data_sel_4_few_shot_examples.text_id==666]

In [ ]:
# @title
# data_sel_4_few_shot_examples[data_sel_4_few_shot_examples.text_id==2729]

In [ ]:
# @title
# data_sel_4_few_shot_examples

In [ ]:
data_sel = data

In [ ]:
import glob

files = glob.glob(output_data_path + "/*.pkl")

results = []
for file in files:
    res = pd.read_pickle(file)

    # try:
    #     if len(res[3]['choices'])>1:
    #         raise Exception('ERROR: Multilabel detected!')
    #         break
    #     sdg_pred = res[3]['choices'][0]['message']['content']
    #     res4df = (res[0], res[1], res[2], sdg_pred)
    #     print(res4df)
    #     tuple_list.append(res4df)
    # except:
    #     print("#################################################################")
    #     print(res)
    #     print("#################################################################")

    # print(res)
    results.append(res)
    # break

len(results)

In [ ]:
results_processed = pd.DataFrame(results, columns=['cnt', 'text_id', 'sdg_true', 'result'])
results_processed.head(3)

In [ ]:
results_processed['label'] = results_processed['result'].str.extract(r'SDG (\d+)')
results_processed.head(3)

In [ ]:
final = data_sel.merge(results_processed, left_index=True, right_index=True)
final.head(3)

In [ ]:
final.to_excel(output_data_path+f"/{experiment_name}.xlsx")

In [ ]:
final  = final[~final.label.isna()]

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

y_true = final.sdg
y_pred = final.label
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

In [ ]:
classification_report_filename = output_data_path+f'/{experiment_name}_classification_report.xlsx'
classification_report_filename

In [ ]:
report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df.to_excel(classification_report_filename)

In [ ]:
test = pd.read_excel(classification_report_filename)
test